Takeaways
- Need new auth to google sheets to upload these averages and use to base on weekly trades on M1 Finance
- Try to consolidate scripts into a function instead that can be reused depending on the ticker


In [42]:
import pandas as pd
import yfinance as yf
import os

pwd = os.getcwd()

In [18]:
mcd = yf.Ticker("mcd")
mcd_history = mcd.history(period="max")

# Create Ticker Column
mcd_history["Ticker"] = "MCD"

# Create rolling weekly average column
mcd_history["weekly_avg_close"] = mcd_history["Close"].rolling(7).mean()

# Create weekly average close percent column
mcd_history["weekly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["weekly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling monthly average column
mcd_history["monthly_avg_close"] = mcd_history["Close"].rolling(30).mean()

# Create monthly average close percent column
mcd_history["monthly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["monthly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling semi_annual average column
mcd_history["semi_annual_avg_close"] = mcd_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
mcd_history["semi_annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["semi_annual_avg_close"]) / mcd_history["Close"] * 100

# Create rolling annual average column
mcd_history["annual_avg_close"] = mcd_history["Close"].rolling(365).mean()

# Create annual average close percent column
mcd_history["annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["annual_avg_close"]) / mcd_history["Close"] * 100


# Create rolling two_year average column
mcd_history["two_year_avg_close"] = mcd_history["Close"].rolling(730).mean()

# Create two_year average close percent column
mcd_history["two_year_avg_close_percent"] = (mcd_history["Close"] - mcd_history["two_year_avg_close"]) / mcd_history["Close"] * 100

# Drop NAs
mcd_history = mcd_history.dropna()

# Grab only needed columns
mcd_price_action = mcd_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
mcd_price_action = mcd_price_action.reset_index()

# Export to CSV
mcd_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [13]:
pep = yf.Ticker("PEP")
pep_history = pep.history(period="max")

# Create Ticker Column
pep_history["Ticker"] = "PEP"

# Create rolling weekly average column
pep_history["weekly_avg_close"] = pep_history["Close"].rolling(7).mean()

# Create weekly average close percent column
pep_history["weekly_avg_close_percent"] = (pep_history["Close"] - pep_history["weekly_avg_close"]) / pep_history["Close"] * 100

# Create rolling monthly average column
pep_history["monthly_avg_close"] = pep_history["Close"].rolling(30).mean()

# Create monthly average close percent column
pep_history["monthly_avg_close_percent"] = (pep_history["Close"] - pep_history["monthly_avg_close"]) / pep_history["Close"] * 100

# Create rolling semi_annual average column
pep_history["semi_annual_avg_close"] = pep_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
pep_history["semi_annual_avg_close_percent"] = (pep_history["Close"] - pep_history["semi_annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling annual average column
pep_history["annual_avg_close"] = pep_history["Close"].rolling(365).mean()

# Create annual average close percent column
pep_history["annual_avg_close_percent"] = (pep_history["Close"] - pep_history["annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling two_year average column
pep_history["two_year_avg_close"] = pep_history["Close"].rolling(730).mean()

# Create two_year average close percent column
pep_history["two_year_avg_close_percent"] = (pep_history["Close"] - pep_history["two_year_avg_close"]) / pep_history["Close"] * 100


# Drop NAs
pep_history = pep_history.dropna()

# Grab only needed columns
pep_price_action = pep_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
pep_price_action = pep_price_action.reset_index()


# Export to CSV
pep_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

,Date,Ticker,Close,weekly_avg_close_percent,monthly_avg_close_percent,semi_annual_avg_close_percent,annual_avg_close_percent,two_year_avg_close_percent,weekly_avg_close,monthly_avg_close,semi_annual_avg_close,annual_avg_close,two_year_avg_close
0,2022-11-11 00:00:00-05:00,PEP,178.050003,-0.756609,2.283442,5.338830,8.846193,18.002854,179.397145,173.984334,168.544216,162.299356,145.995921
1,2022-11-10 00:00:00-05:00,PEP,181.699997,1.252454,4.517701,7.279675,10.730490,19.689055,179.424288,173.491334,168.472828,162.202696,145.924985
2,2022-11-09 00:00:00-05:00,PEP,178.550003,-0.414449,3.114908,5.696968,9.216106,18.315086,179.290002,172.988334,168.378067,162.094646,145.848416
3,2022-11-08 00:00:00-05:00,PEP,180.710007,0.546261,4.455208,6.869917,10.357109,19.331299,179.722859,172.659001,168.295378,161.993675,145.776414
4,2022-11-07 00:00:00-05:00,PEP,180.210007,0.149829,4.462020,6.665296,10.167252,19.148451,179.940000,172.169000,168.198476,161.887601,145.702582


In [14]:
msft = yf.Ticker("MSFT")
msft_history = msft.history(period="max")

msft_history["Ticker"] = "MSFT"

# Create rolling weekly average column
msft_history["weekly_avg_close"] = msft_history["Close"].rolling(7).mean()

# Create weekly average close percent column
msft_history["weekly_avg_close_percent"] = (msft_history["Close"] - msft_history["weekly_avg_close"]) / msft_history["Close"] * 100

# Create rolling monthly average column
msft_history["monthly_avg_close"] = msft_history["Close"].rolling(30).mean()

# Create monthly average close percent column
msft_history["monthly_avg_close_percent"] = (msft_history["Close"] - msft_history["monthly_avg_close"]) / msft_history["Close"] * 100

# Create rolling semi_annual average column
msft_history["semi_annual_avg_close"] = msft_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
msft_history["semi_annual_avg_close_percent"] = (msft_history["Close"] - msft_history["semi_annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling annual average column
msft_history["annual_avg_close"] = msft_history["Close"].rolling(365).mean()

# Create annual average close percent column
msft_history["annual_avg_close_percent"] = (msft_history["Close"] - msft_history["annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling two_year average column
msft_history["two_year_avg_close"] = msft_history["Close"].rolling(730).mean()

# Create two_year average close percent column
msft_history["two_year_avg_close_percent"] = (msft_history["Close"] - msft_history["two_year_avg_close"]) / msft_history["Close"] * 100


# Drop NAs
msft_history = msft_history.dropna()

# Grab only needed columns
msft_price_action = msft_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
msft_price_action = msft_price_action.reset_index()

# Export to CSV
msft_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

,Date,Ticker,Close,weekly_avg_close_percent,monthly_avg_close_percent,semi_annual_avg_close_percent,annual_avg_close_percent,two_year_avg_close_percent,weekly_avg_close,monthly_avg_close,semi_annual_avg_close,annual_avg_close,two_year_avg_close
0,2022-11-11 00:00:00-05:00,MSFT,247.110001,7.098633,5.129025,-7.520840,-14.403782,1.863485,229.568569,234.435666,265.694748,282.703187,242.505143
1,2022-11-10 00:00:00-05:00,MSFT,242.979996,7.107580,3.711415,-9.436863,-16.343934,0.249159,225.709998,233.962000,265.909684,282.692485,242.374589
2,2022-11-09 00:00:00-05:00,MSFT,224.509995,0.407871,-4.128697,-18.546318,-25.917517,-7.900721,223.594284,233.779333,266.148332,282.697410,242.247902
3,2022-11-08 00:00:00-05:00,MSFT,228.869995,1.829483,-2.386219,-16.443023,-23.543153,-5.801088,224.682857,234.331333,266.503141,282.753208,242.146944
4,2022-11-07 00:00:00-05:00,MSFT,227.869995,0.959818,-2.945834,-17.120218,-24.107276,-6.218958,225.682857,234.582667,266.881836,282.803244,242.041135


Trying to create reusable functions

In [89]:
# ticker_symbol = "MCD"

def ticker_function(ticker_symbol):

    ticker = yf.Ticker(ticker_symbol)
    ticker_history = ticker.history(period="max")

    ticker_history["Ticker"] = ticker_symbol

    ticker_history["Wkly_Avg_Close"] = ticker_history["Close"].rolling(7).mean()
    ticker_history["Wkly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Wkly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Monthly_Avg_Close"] = ticker_history["Close"].rolling(30).mean()
    ticker_history["Monthly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Monthly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Semi_Annual_Avg_Close"] = ticker_history["Close"].rolling(183).mean()
    ticker_history["Semi_Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Semi_Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Annual_Avg_Close"] = ticker_history["Close"].rolling(365).mean()
    ticker_history["Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Two_Year_Avg_Close"] = ticker_history["Close"].rolling(730).mean()
    ticker_history["Two_Year_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Two_Year_Avg_Close"]) / ticker_history["Close"] * 100

    ticker_history = ticker_history.dropna()

    ticker_price_action = ticker_history[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(5)
    ticker_price_action = ticker_price_action.reset_index()

    return ticker_price_action

ticker_function("MCD")
ticker_function("PEP")

,Date,Ticker,Close,Wkly_Avg_Close_Percent,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close_Percent,Two_Year_Avg_Close_Percent,Wkly_Avg_Close,Monthly_Avg_Close,Semi_Annual_Avg_Close,Annual_Avg_Close,Two_Year_Avg_Close
0,2022-11-11 00:00:00-05:00,PEP,178.050003,-0.756609,2.283442,5.338830,8.846193,18.002854,179.397145,173.984334,168.544217,162.299356,145.995921
1,2022-11-10 00:00:00-05:00,PEP,181.699997,1.252454,4.517701,7.279675,10.730490,19.689055,179.424288,173.491334,168.472828,162.202696,145.924985
2,2022-11-09 00:00:00-05:00,PEP,178.550003,-0.414449,3.114908,5.696967,9.216106,18.315086,179.290002,172.988334,168.378067,162.094646,145.848416
3,2022-11-08 00:00:00-05:00,PEP,180.710007,0.546261,4.455208,6.869917,10.357109,19.331299,179.722859,172.659001,168.295378,161.993675,145.776415
4,2022-11-07 00:00:00-05:00,PEP,180.210007,0.149829,4.462020,6.665296,10.167253,19.148451,179.940000,172.169000,168.198476,161.887600,145.702583


Playing around with multiple tickers variable and classes

In [74]:
from yfinance import Ticker


class TickerRegistry:

    # ticker_symbol = { }
    # tickers = yf.Tickers("mcd pep msft")

    def __init__(self, ticker):
        self.ticker = ticker

    def add_ticker_info(self):
        tickers = yf.Tickers("mcd pep msft")
        return tickers.tickers["MCD"].history()

    def create_columns(self):
        msft_history["Ticker"] = "MSFT"



mcd_ticker = TickerRegistry("MCD")
mcd_ticker.add_ticker_info()

print(mcd_ticker.add_ticker_info())



                                 Open        High         Low       Close  \
Date                                                                        
2022-10-12 00:00:00-04:00  237.270004  240.039993  237.059998  237.059998   
2022-10-13 00:00:00-04:00  234.460007  249.300003  234.460007  246.750000   
2022-10-14 00:00:00-04:00  249.080002  249.429993  242.410004  243.160004   
2022-10-17 00:00:00-04:00  245.800003  247.220001  243.529999  246.479996   
2022-10-18 00:00:00-04:00  249.369995  250.419998  247.250000  249.190002   
2022-10-19 00:00:00-04:00  250.000000  252.520004  249.119995  250.750000   
2022-10-20 00:00:00-04:00  249.550003  251.750000  249.050003  249.770004   
2022-10-21 00:00:00-04:00  250.210007  255.360001  249.500000  254.550003   
2022-10-24 00:00:00-04:00  255.279999  256.869995  253.070007  255.320007   
2022-10-25 00:00:00-04:00  254.570007  257.279999  254.399994  255.729996   
2022-10-26 00:00:00-04:00  257.839996  260.690002  256.119995  256.609985   